In [1]:
import math
import numpy as np 
import matplotlib.pyplot as plt 
from numpy import random 

In [2]:
#initialize the values
N = 100
D = 1
dx = 1/N
w = 1.9
e = 10**(-5)
eta = 1

In [5]:
def update_SOR(Y, w):
    '''
    The SOR method grid update function
    Args:
        -Y, the grid at the current timestep
        -w, the omega constant
        -N, the size of the grid
    Out:
        -Y, the grid at the next timestep
    '''
    #update the grid at position i, j and take boundary conditions into account
    for j in range(1, len(Y)-1):
        for i in range(len(Y)):
            if i == 0:
                 Y[j,i] = (1-w)*Y[j,i] + (w/4) * (Y[j+1,i] + Y[j-1,i] + Y[j,i+1] + Y[j,-1]) 
            elif i == N-1:
                 Y[j,i] = (1-w)*Y[j,i] + (w/4) * (Y[j+1,i] + Y[j-1,i] + Y[j,0] + Y[j,i-1]) 
            else:
                 Y[j,i] = (1-w)*Y[j,i] + (w/4) * (Y[j+1,i] + Y[j-1,i] + Y[j,i+1] + Y[j,i-1])  
    return(Y)  

In [6]:
def init(N):
    '''
    Initialize a grid with all zeros except on the first row where the value is all ones
    Args:
        -N the size of the grid
    Out: the initial state of the grid
    '''
    k = 1
    Y = np.zeros((N,N))
    Y[0] = np.ones(N)
    Y_prev = np.copy(Y)
    Y = update_SOR(Y, w, N)
    #calculate the difference between the last two iterations
    diff = np.abs(Y - Y_prev)
    diff_val = (np.amax(diff))
    #until the differnce is smaller than epsilon, update the grid
    while(diff_val > e):
        k = k+1
        Y_prev = np.copy(Y)
        Y = update_SOR(Y, w, N)
        diff = np.abs(Y - Y_prev)
        diff_val = (np.amax(diff))
    return Y
Y = init(N)

In [ ]:
def neighbours_function(new_s, Y, neighbours):
    if len(neighbours) != 0:
        if int(new_s[0]) == 0:
            if int(new_s[1]) ==0:
                if Y[int(new_s[0]),-1]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),-1]])
                if Y[int(new_s[0]),1]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),1]])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0])+1,int(new_s[1])]])
            elif int(new_s[1]) == len(Y):
                if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),int(new_s[1])-1]])
                if Y[int(new_s[0]),0]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),0]])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0])+1,int(new_s[1])]])            
            else:
                if Y[int(new_s[0]),int(new_s[1])+1]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),int(new_s[1])+1]])
                if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),int(new_s[1])-1]])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0])+1,int(new_s[1])]])
        elif int(new_s[1]) == 0:
            if Y[int(new_s[0]),-1]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0]),-1]])
            if Y[int(new_s[0]),1]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0]),1]])  
            if Y[int(new_s[0])-1,int(new_s[1])]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0])-1,int(new_s[1])]])
            if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0])+1,int(new_s[1])]])
        elif int(new_s[1]) == len(Y):
            if Y[int(new_s[0]),0]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0]),0]])
            if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0]),int(new_s[1])-1]])  
            if Y[int(new_s[0])-1,int(new_s[1])]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0])-1,int(new_s[1])]])
            if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                neighbours = np.vstack([neighbours, [int(new_s[0])+1,int(new_s[1])]])           
        else:
            for i in [-1,1]:
                if Y[int(new_s[0])+i,int(new_s[1])]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0])+i,int(new_s[1])] ])
                if Y[int(new_s[0]),int(new_s[1])+i]!=0:
                    neighbours = np.vstack([neighbours, [int(new_s[0]),int(new_s[1])+i]] )
    else: 
        if int(new_s[0]) == 0:
            if int(new_s[1]) ==0:
                if Y[int(new_s[0]),-1]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),-1])
                if Y[int(new_s[0]),1]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),1])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0])+1,int(new_s[1])])
            elif int(new_s[1]) == len(Y):
                if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),int(new_s[1])-1])
                if Y[int(new_s[0]),0]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),0])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0])+1,int(new_s[1])])            
            else:
                if Y[int(new_s[0]),int(new_s[1])+1]!=0:
                    neighbours = np.append(neighbours,  [int(new_s[0]),int(new_s[1])+1])
                if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                    neighbours = np.append(neighbours,  [int(new_s[0]),int(new_s[1])-1])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0])+1,int(new_s[1])])
        elif int(new_s[0]) == len(Y)-1:
            if int(new_s[1]) ==0:
                if Y[int(new_s[0]),-1]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),-1])
                if Y[int(new_s[0]),1]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),1])  
                if Y[int(new_s[0])-1,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])
            elif int(new_s[1]) == len(Y):
                if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),int(new_s[1])-1])
                if Y[int(new_s[0]),0]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0]),0])  
                if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])            
            else:
                if Y[int(new_s[0]),int(new_s[1])+1]!=0:
                    neighbours = np.append(neighbours,  [int(new_s[0]),int(new_s[1])+1])
                if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                    neighbours = np.append(neighbours,  [int(new_s[0]),int(new_s[1])-1])  
                if Y[int(new_s[0])-1,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])
        elif int(new_s[1]) == 0:
            if Y[int(new_s[0]),-1]!=0:
                neighbours = np.append(neighbours, [int(new_s[0]),-1])
            if Y[int(new_s[0]),1]!=0:
                neighbours = np.append(neighbours, [int(new_s[0]),1])  
            if Y[int(new_s[0])-1,int(new_s[1])]!=0:
                neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])
            if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])
        elif int(new_s[1]) == len(Y):
            if Y[int(new_s[0]),int(new_s[1])-1]!=0:
                neighbours = np.append(neighbours, [int(new_s[0]),int(new_s[1])-1])
            if Y[int(new_s[0]),0]!=0:
                neighbours = np.append(neighbours, [int(new_s[0]),0])  
            if Y[int(new_s[0])-1,int(new_s[1])]!=0:
                neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])
            if Y[int(new_s[0])+1,int(new_s[1])]!=0:
                neighbours = np.append(neighbours, [int(new_s[0])-1,int(new_s[1])])
        else:
            for i in [-1,1]:
                if Y[int(new_s[0])+i,int(new_s[1])]!=0:
                    neighbours = np.append(neighbours,  [int(new_s[0])+i,int(new_s[1])])
                if Y[int(new_s[0]),int(new_s[1])+i]!=0:
                    neighbours = np.append(neighbours,  [int(new_s[0]),int(new_s[1])+i] )
        neighbours = np.array([neighbours])
    return neighbours


In [39]:
def growth(Y, seed, neighbours, eta):
    total = 0
    prop = []
    for s in seed:
        Y[int(s[0]),int(s[1])] = 0
    for n in neighbours:
        total += (Y[int(n[0]),int(n[1])])**eta
        c_ij = (Y[int(n[0]),int(n[1])])**eta
        prop.append(c_ij)
    prop = prop/total
    
    index = random.choice(np.arange(len(neighbours)), p = prop)
    new_s = neighbours[index] 
    
    seed = np.vstack([seed, new_s])
    neighbours = np.delete(neighbours, index, axis = 0)
    neighbours =  neighbours_function(new_s, Y, neighbours)
    return Y, seed, neighbours

In [41]:
def itter(seed, Y, eta, max_iter):
    neighbours =  np.array([])
    neighbours = neighbours_function(seed[0], Y, neighbours)
    for i in range(max_iter):
        Y, seed, neighbours = growth(Y, seed, neighbours, eta)
        Y = update_SOR(Y, w,N)
    plt.imshow(Y)
    plt.show()

In [42]:
seed = np.array([[99,50]])
neighbours= np.array([[98,50], [99,49], [99,51]])
itter(seed, Y, eta, 100)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 6